In [2]:
import sqlite3

sl_conn = sqlite3.connect('test_fire_db.sqlite3')
sl_curs = sl_conn.cursor()

In [64]:
# try a table representation
# next create table in postgresql
# make a schema

create_modus_table = """
CREATE TABLE modus(
    id INTEGER PRIMARY KEY,
    latitude NUMERIC(6,3),
    longitude NUMERIC(6,3),
    brightness NUMERIC(4,1),
    scan NUMERIC(2,1)
)
"""

drop_table = """
DROP TABLE modus
"""

In [67]:
sl_curs.execute(create_modus_table)
# drop table to reinitializie
# sl_curs.execute(drop_table)
# sl_curs.commit()

In [7]:
# weird that worked...
# now try to put in a dataframe row?
import pandas as pd

url = 'https://firms.modaps.eosdis.nasa.gov/data/active_fire/c6/csv/MODIS_C6_USA_contiguous_and_Hawaii_24h.csv'
df = pd.read_csv(url)

In [8]:
df.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,confidence,version,bright_t31,frp,daynight
0,18.924,-70.361,301.3,2.2,1.4,2019-09-02,325,T,40,6.0NRT,290.6,10.2,N
1,33.021,-86.008,313.4,1.6,1.2,2019-09-02,330,T,87,6.0NRT,290.0,23.4,N
2,33.024,-85.998,308.2,1.6,1.2,2019-09-02,330,T,74,6.0NRT,290.4,15.6,N
3,33.021,-86.014,305.5,1.6,1.2,2019-09-02,330,T,64,6.0NRT,290.1,12.1,N
4,37.238,-82.041,300.6,1.0,1.0,2019-09-02,330,T,30,6.0NRT,288.7,4.5,N


In [27]:
df.loc[0, target_columns]

latitude      18.924
longitude    -70.361
brightness     301.3
scan             2.2
Name: 0, dtype: object

In [58]:
target_columns = ['latitude', 'longitude', 'brightness', 'scan']

insert1 = tuple(x for x in df.loc[0, target_columns])

# for x in df.loc[0, target_columns]:
#     print(x)
insert1

(18.924, -70.361, 301.3, 2.2)

In [70]:
attempted_insert = """
    INSERT INTO modus (latitude, longitude, brightness, scan)
    VALUES """+ str(insert1)
attempted_insert

'\n    INSERT INTO modus (latitude, longitude, brightness, scan)\n    VALUES (18.924, -70.361, 301.3, 2.2)'

In [72]:
# test the insert
sl_curs.execute(attempted_insert)

In [84]:
sl_curs.execute("SELECT * FROM modus")

In [85]:
rows = sl_curs.fetchall()
 
for row in rows:
    print(row)

In [86]:
# # reset db, don't do this at home
# sl_curs.execute("""
# DELETE FROM modus 
# """)

In [96]:
# now I will load our entire dataframe into the sqlite3 db
for row in df[target_columns].values:
    attempted_insert = """
    INSERT INTO modus (latitude, longitude, brightness, scan)
    VALUES """+ str(tuple(row))
    sl_curs.execute(attempted_insert)

In [100]:
# check results
sl_curs.execute("SELECT * FROM modus LIMIT 10")
rows = sl_curs.fetchall()
 
for row in rows:
    print(row)

(1, 18.924, -70.361, 301.3, 2.2)
(2, 33.021, -86.008, 313.4, 1.6)
(3, 33.024, -85.99799999999999, 308.2, 1.6)
(4, 33.021, -86.014, 305.5, 1.6)
(5, 37.238, -82.041, 300.6, 1)
(6, 36.804, -84.78299999999999, 304.9, 1.2)
(7, 43.519, -89.29299999999999, 300.5, 1.4)
(8, 41.901, -96.321, 303, 3.9)
(9, 42.294, -106.87100000000001, 321.2, 1)
(10, 42.292, -106.883, 332.4, 1)
